<a href="https://colab.research.google.com/github/atg2431-alpha/rag_using_chromadb/blob/main/amazon_q_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain_groq chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 25.0 MB/s eta 0:

In [3]:
import pandas as pd
import numpy as np
import torch
from langchain_groq import ChatGroq

In [4]:
from google.colab import userdata
key = userdata.get('groq_amazon')

In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model = 'mixtral-8x7b-32768',
    groq_api_key=key,
    temperature=0,
    max_tokens = None,
    timeout = None,
    max_retries=2
)

In [6]:
llm.invoke("Hi")

AIMessage(content="Hello! It's nice to meet you. Is there something specific you would like to know or discuss about computer science, programming, or data science? I'm here to help answer any questions you have. Just let me know!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 9, 'total_tokens': 58, 'completion_time': 0.077426167, 'prompt_time': 0.001840068, 'queue_time': 0.012495962, 'total_time': 0.079266235}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-598be1e3-6668-4e6e-99f5-cc7182cdb2d3-0', usage_metadata={'input_tokens': 9, 'output_tokens': 49, 'total_tokens': 58})

In [35]:
df = pd.read_csv('amazon_dataset.csv')
df.drop(columns = ['flags','intent','category'],inplace =True)
df.head()

,instruction,response
0,question about cancelling order {{Order Number}},I've understood you have a question regarding ...
1,i have a question about cancelling oorder {{Or...,I've been informed that you have a question ab...
2,i need help cancelling puchase {{Order Number}},I can sense that you're seeking assistance wit...
3,I need to cancel purchase {{Order Number}},I understood that you need assistance with can...
4,"I cannot afford this order, cancel purchase {{...",I'm sensitive to the fact that you're facing f...


In [36]:
df = df.sample(1000)

In [8]:
import chromadb
from transformers import AutoTokenizer, AutoModel

client = chromadb.Client()
collection = client.create_collection(name="amazon_qa")

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [9]:
def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings[0].numpy()

In [38]:
all_ids = collection.get()["ids"]
collection.delete(all_ids)

In [39]:
import torch
for i, row in df.iterrows():
    embedding = embed_text(row['instruction'])
    collection.add(
        embeddings=[embedding],
        metadatas=[{"instruction": row['instruction'], "response": row['response']}],
        ids=[str(i)]
    )

In [52]:
query = "How do I cancel my order?"
query_embedding = embed_text(query)

results = collection.query(query_embeddings=[query_embedding], n_results=2)

In [53]:
results

{'ids': [['783', '583']],
 'embeddings': None,
 'documents': [[None, None]],
 'uris': None,
 'data': None,
 'metadatas': [[{'instruction': 'help me to cancel purchase {{Order Number}}',
    'response': "I've noticed that you need assistance with canceling your purchase of the product associated with the order number {{Order Number}}. I apologize for any inconvenience this may have caused you. To proceed with the cancellation, please follow these steps:\n\n1. Sign in to your account: Access our platform by logging into your {{Online Company Portal Info}}.\n2. Navigate to your orders: Once you're signed in, go to the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section.\n3. Locate your purchase: Look for the order with the number {{Order Number}} and click on it.\n4. Initiate the cancellation: Within the order details, you should find an option to '{{Online Order Interaction}}'. Please select this to proceed.\n5. Confirm the cancellation: The system may prompt you to 

In [41]:
results['metadatas'][0][0]

{'instruction': 'help me to cancel purchase {{Order Number}}',
 'response': "I've noticed that you need assistance with canceling your purchase of the product associated with the order number {{Order Number}}. I apologize for any inconvenience this may have caused you. To proceed with the cancellation, please follow these steps:\n\n1. Sign in to your account: Access our platform by logging into your {{Online Company Portal Info}}.\n2. Navigate to your orders: Once you're signed in, go to the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section.\n3. Locate your purchase: Look for the order with the number {{Order Number}} and click on it.\n4. Initiate the cancellation: Within the order details, you should find an option to '{{Online Order Interaction}}'. Please select this to proceed.\n5. Confirm the cancellation: The system may prompt you to confirm your cancellation request. Please follow the instructions and provide any required feedback if necessary.\n\nIf you en

USING CHAIN

In [45]:
from langchain_core.prompts import PromptTemplate
from langchain import LLMChain

prompt_template = """
You are an intelligent assistant that helps users by providing clear and accurate answers based on available information.

**User Query:**
{query}

**Retrieved Information:**
{retrieved_response}

Using the information above, please provide a helpful and concise answer to the user's query.
"""

prompt_extract = PromptTemplate(
    input_variables=["query", "retrieved_response"],
    template=prompt_template
)

chain = LLMChain(llm=llm, prompt=prompt_extract)

<ipython-input-45-bbc4e859802b>:21: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_extract)


In [50]:
def get_response(user_query):
    results = collection.query(
        query_texts=[user_query],
        n_results=1
    )

    if results['metadatas']:
        retrieved_response = results['metadatas'][0][0]['response']  # Adjust indexing based on your data structure
    else:
        retrieved_response = "I'm sorry, I couldn't find relevant information to answer your question."

    final_response = chain.run({
        "query": user_query,
        "retrieved_response": retrieved_response
    })

    return final_response

In [62]:
user_query = "How to cancel my order?"
response = get_response(user_query)
print(response)

I'm sorry for any confusion, it seems the information I have is about downloading an invoice, not canceling an order. 

Unfortunately, I can't assist you with canceling your order as I don't have the necessary information or the capability to interact with your account on our website. 

Here's what you can do:

1. Log in to your account on our website.
2. Navigate to the 'Orders' section.
3. Look for the order you want to cancel.
4. There should be an option to cancel the order, usually represented by a 'Cancel' button or link.

If you can't find this option or have further questions, I would recommend reaching out to our customer support team for further assistance. They have the tools and knowledge to help you with your order cancellation.


USING PIPELINE

In [54]:
pipe = prompt_extract | llm

In [56]:
def get_response2(user_query):
    """
    Fetches the most relevant response from ChromaDB and uses the LLM to generate a refined answer.

    Parameters:
    - user_query (str): The question posed by the user.

    Returns:
    - str: The final answer generated by the LLM.
    """
    results = collection.query(
        query_texts=[user_query],
        n_results=1
    )

    if results['metadatas']:
        retrieved_response = results['metadatas'][0][0]['response']  # Adjust indexing based on your data structure
    else:
        retrieved_response = "I'm sorry, I couldn't find relevant information to answer your question."

    final_response = pipe.invoke(input = {
        "query": user_query,
        "retrieved_response": retrieved_response
    })

    return final_response

In [60]:
user_query = "How to cancel my order?"
response = get_response2(user_query)
response.content

"I'm sorry for any confusion, it seems the information I have is about downloading an invoice, not canceling an order. \n\nUnfortunately, I can't assist you with canceling your order as I don't have the necessary information or the capability to interact with your account on our website. \n\nHere's what you can do:\n\n1. Log in to your account on our website.\n2. Navigate to the 'Orders' section.\n3. Look for the order you want to cancel.\n4. There should be an option to cancel the order, usually represented by a 'Cancel' button or link.\n\nIf you can't find this option or have further questions, I would recommend reaching out to our customer support team for further assistance. They have the tools and knowledge to help you with your order cancellation."